In [ ]:
from os import listdir
from os.path import isfile, join
import subprocess
import matplotlib.pyplot as plt
import numpy as np
import os
files = [f for f in listdir('../acs_mosaic_2.0') if isfile(join('../acs_mosaic_2.0', f))]

In [ ]:
i=2
ras = []
decs = []
for f in files:
    ra, dec = f.split('_')[1:3]
    if f.split('_')[-1] == 'sci.fits':
        #print(ra, dec)
        ras.append(np.float(ra))
        decs.append(np.float(dec))
        i+=1
plt.figure(figsize = (15,15))
plt.plot(np.array(ras), np.array(decs), 'o')
plt.show()
print(i-2)